In [1]:
# coding: utf-8
import os

import pandas as pd
import numpy as np
import scipy
import lightgbm as lgb
from sklearn import metrics
from sklearn import model_selection
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV
from sklearn import preprocessing
from sklearn.decomposition import PCA, TruncatedSVD
import matplotlib.pyplot as plt
import seaborn as sns

from mydatools.plot import plot_multiclass_feature_dist
from mydatools.features_analyze import get_top_k_corr

% matplotlib inline

## Config

In [2]:
trn_path = './data/input/application_train.csv'
tst_path = './data/input/application_test.csv'
id_col = 'SK_ID_CURR'
label_col = 'TARGET'

submission_path = './data/output/submission/submission.csv'
output_id_col = id_col
output_label_col = label_col

## Load Data

In [3]:
trn_df = pd.read_csv(trn_path)
trn_df['ds_type'] = 'trn'
trn_df[label_col] = trn_df[label_col].astype('int')

tst_df = pd.read_csv(tst_path)
tst_df['ds_type'] = 'tst'

full_df = pd.concat([trn_df, tst_df])

del(trn_df)
del(tst_df)

full_df.head()

,AMT_ANNUITY,AMT_CREDIT,AMT_GOODS_PRICE,AMT_INCOME_TOTAL,AMT_REQ_CREDIT_BUREAU_DAY,AMT_REQ_CREDIT_BUREAU_HOUR,AMT_REQ_CREDIT_BUREAU_MON,AMT_REQ_CREDIT_BUREAU_QRT,AMT_REQ_CREDIT_BUREAU_WEEK,AMT_REQ_CREDIT_BUREAU_YEAR,...,TOTALAREA_MODE,WALLSMATERIAL_MODE,WEEKDAY_APPR_PROCESS_START,YEARS_BEGINEXPLUATATION_AVG,YEARS_BEGINEXPLUATATION_MEDI,YEARS_BEGINEXPLUATATION_MODE,YEARS_BUILD_AVG,YEARS_BUILD_MEDI,YEARS_BUILD_MODE,ds_type
0,24700.5,406597.5,351000.0,202500.0,0.0,0.0,0.0,0.0,0.0,1.0,...,0.0149,"Stone, brick",WEDNESDAY,0.9722,0.9722,0.9722,0.6192,0.6243,0.6341,trn
1,35698.5,1293502.5,1129500.0,270000.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0714,Block,MONDAY,0.9851,0.9851,0.9851,0.7960,0.7987,0.8040,trn
2,6750.0,135000.0,135000.0,67500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,MONDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
3,29686.5,312682.5,297000.0,135000.0,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,WEDNESDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn
4,21865.5,513000.0,513000.0,121500.0,0.0,0.0,0.0,0.0,0.0,0.0,...,NaN,NaN,THURSDAY,NaN,NaN,NaN,NaN,NaN,NaN,trn


In [4]:
# dataset type
is_trn = full_df['ds_type'] == 'trn'
is_tst = full_df['ds_type'] == 'tst'

## Features

In [5]:
feature_columns = []

def add_features(features):
    if not isinstance(features, list):
        features = [features]
    global feature_columns
    feature_columns.extend([f for f in features if f not in feature_columns])

**numerical_features**

In [6]:
numerical_features = full_df.dtypes[full_df.dtypes != 'object'].index.tolist()
numerical_features = [c for c in numerical_features if c not in [id_col, label_col, 'ds_type']]
add_features(numerical_features)

**categorical_features**

In [7]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
full_df[categorical_features].nunique()

CODE_GENDER                    3
EMERGENCYSTATE_MODE            2
FLAG_OWN_CAR                   2
FLAG_OWN_REALTY                2
FONDKAPREMONT_MODE             4
HOUSETYPE_MODE                 3
NAME_CONTRACT_TYPE             2
NAME_EDUCATION_TYPE            5
NAME_FAMILY_STATUS             6
NAME_HOUSING_TYPE              6
NAME_INCOME_TYPE               8
NAME_TYPE_SUITE                7
OCCUPATION_TYPE               18
ORGANIZATION_TYPE             58
WALLSMATERIAL_MODE             7
WEEKDAY_APPR_PROCESS_START     7
ds_type                        2
dtype: int64

In [8]:
categorical_features = full_df.dtypes[full_df.dtypes == 'object'].index.tolist()
categorical_features = categorical_features[:-1] # 为了去掉ds_type
# 去掉分类数太多
# categorical_features = full_df[categorical_features].columns[full_df[categorical_features].nunique() <= 10]

# get dummies
full_df = pd.get_dummies(full_df, columns=categorical_features, prefix_sep=':')

# add features
new_features = full_df.columns.tolist()[full_df.columns.tolist().index('ds_type') + 1:]
add_features(new_features)

### add extra data

In [9]:
def summary_extra_data(df, suffix):
    # 分类特征get_dummies
    df = pd.get_dummies(df, columns=df.columns[df.dtypes == 'object'].tolist())
    df = df.fillna(0)

    # 求平均
    df = pd.concat([
        df.groupby('SK_ID_CURR').mean(),
        df.groupby('SK_ID_CURR').size().rename('cnt'),
    ], axis=1)

    # 给特征加上前缀
    df.columns = [suffix+f for f in df.columns.tolist()]

    return df

In [10]:
# bureau = pd.read_csv('./data/input/bureau.csv')
# test_df = bureau.iloc[:100, :5]
# test_df.head()

# summary_extra_data(test_df, 'test')

**bureau.csv**

In [11]:
bureau = pd.read_csv('./data/input/bureau.csv')
bureau = bureau.drop('SK_ID_BUREAU', axis=1)

bureau = summary_extra_data(bureau, 'bureau_')

full_df = full_df.join(bureau, on='SK_ID_CURR')
add_features(bureau.columns.tolist())

**previous_application.csv**

In [12]:
prevapp = pd.read_csv('./data/input/previous_application.csv')
prevapp = prevapp.drop('SK_ID_PREV', axis=1)

prevapp = summary_extra_data(prevapp, 'prevapp_')

full_df = full_df.join(prevapp, on='SK_ID_CURR')
add_features(prevapp.columns.tolist())

**POS_CASH_balance.csv**

In [13]:
pcblc = pd.read_csv('./data/input/POS_CASH_balance.csv')
pcblc = pcblc.drop(['SK_ID_PREV', 'MONTHS_BALANCE'], axis=1)

pcblc = summary_extra_data(pcblc, 'pcblc_')

full_df = full_df.join(pcblc, on='SK_ID_CURR')
add_features(pcblc.columns.tolist())

**credit_card_balance.csv**

In [14]:
ccblc = pd.read_csv('./data/input/credit_card_balance.csv')
ccblc = ccblc.drop(['SK_ID_PREV', 'MONTHS_BALANCE'], axis=1)

ccblc = summary_extra_data(ccblc, 'ccblc_')

full_df = full_df.join(ccblc, on='SK_ID_CURR')
add_features(ccblc.columns.tolist())

## LightGBM

In [15]:
trn_df = full_df[is_trn]
tst_df = full_df[is_tst]

X = trn_df[feature_columns]
y = trn_df[label_col]
X_trn, X_val, y_trn, y_val = train_test_split(X, y, test_size=0.20, random_state=20180521)

X_tst = tst_df[feature_columns]

d_trn = lgb.Dataset(X_trn.values, y_trn)
d_val = lgb.Dataset(X_val.values, y_val)

In [16]:
params = {
    'application': 'binary',
    'metric': 'auc',
    'learning_rate': 0.05,
    'max_depth': 4,
    'num_leaves': 20,
    'feature_fraction': 0.8,
    'bagging_fraction': 0.5,
    'bagging_freq': 1,
    'is_unbalance': True,
    'seed': 0,
    'reg_alpha': 0.1,
}
bst = lgb.train(params, d_trn, 500, valid_sets=[d_trn, d_val], early_stopping_rounds=20)

[1]	training's auc: 0.702238	valid_1's auc: 0.703763
Training until validation scores don't improve for 20 rounds.
[2]	training's auc: 0.710525	valid_1's auc: 0.712602
[3]	training's auc: 0.723277	valid_1's auc: 0.724762
[4]	training's auc: 0.725502	valid_1's auc: 0.728132
[5]	training's auc: 0.725115	valid_1's auc: 0.72763
[6]	training's auc: 0.727403	valid_1's auc: 0.729242
[7]	training's auc: 0.727889	valid_1's auc: 0.729676
[8]	training's auc: 0.727348	valid_1's auc: 0.729188
[9]	training's auc: 0.728549	valid_1's auc: 0.730342
[10]	training's auc: 0.731225	valid_1's auc: 0.733193
[11]	training's auc: 0.731294	valid_1's auc: 0.733843
[12]	training's auc: 0.731183	valid_1's auc: 0.733514
[13]	training's auc: 0.731377	valid_1's auc: 0.733672
[14]	training's auc: 0.731463	valid_1's auc: 0.733857
[15]	training's auc: 0.73216	valid_1's auc: 0.734304
[16]	training's auc: 0.732808	valid_1's auc: 0.735051
[17]	training's auc: 0.733205	valid_1's auc: 0.735411
[18]	training's auc: 0.733605	v

[152]	training's auc: 0.785068	valid_1's auc: 0.773868
[153]	training's auc: 0.785143	valid_1's auc: 0.774
[154]	training's auc: 0.785274	valid_1's auc: 0.774066
[155]	training's auc: 0.785366	valid_1's auc: 0.774106
[156]	training's auc: 0.785485	valid_1's auc: 0.774184
[157]	training's auc: 0.785634	valid_1's auc: 0.774255
[158]	training's auc: 0.785798	valid_1's auc: 0.774358
[159]	training's auc: 0.785877	valid_1's auc: 0.774406
[160]	training's auc: 0.786031	valid_1's auc: 0.774399
[161]	training's auc: 0.786231	valid_1's auc: 0.77449
[162]	training's auc: 0.786374	valid_1's auc: 0.774537
[163]	training's auc: 0.786529	valid_1's auc: 0.774593
[164]	training's auc: 0.786633	valid_1's auc: 0.774615
[165]	training's auc: 0.786768	valid_1's auc: 0.774674
[166]	training's auc: 0.786901	valid_1's auc: 0.774744
[167]	training's auc: 0.787052	valid_1's auc: 0.774776
[168]	training's auc: 0.787222	valid_1's auc: 0.774842
[169]	training's auc: 0.787403	valid_1's auc: 0.774932
[170]	training

[302]	training's auc: 0.802837	valid_1's auc: 0.780164
[303]	training's auc: 0.802913	valid_1's auc: 0.780189
[304]	training's auc: 0.802994	valid_1's auc: 0.780253
[305]	training's auc: 0.803089	valid_1's auc: 0.780251
[306]	training's auc: 0.803175	valid_1's auc: 0.780285
[307]	training's auc: 0.803243	valid_1's auc: 0.780288
[308]	training's auc: 0.803336	valid_1's auc: 0.78036
[309]	training's auc: 0.803451	valid_1's auc: 0.780388
[310]	training's auc: 0.803562	valid_1's auc: 0.78042
[311]	training's auc: 0.803667	valid_1's auc: 0.780424
[312]	training's auc: 0.803841	valid_1's auc: 0.780473
[313]	training's auc: 0.803946	valid_1's auc: 0.780483
[314]	training's auc: 0.80402	valid_1's auc: 0.780523
[315]	training's auc: 0.804104	valid_1's auc: 0.780573
[316]	training's auc: 0.804223	valid_1's auc: 0.780629
[317]	training's auc: 0.804342	valid_1's auc: 0.780687
[318]	training's auc: 0.80441	valid_1's auc: 0.780729
[319]	training's auc: 0.804449	valid_1's auc: 0.780749
[320]	training

[452]	training's auc: 0.815457	valid_1's auc: 0.782382
[453]	training's auc: 0.815542	valid_1's auc: 0.782396
[454]	training's auc: 0.815589	valid_1's auc: 0.782409
[455]	training's auc: 0.815653	valid_1's auc: 0.782378
[456]	training's auc: 0.815732	valid_1's auc: 0.782374
[457]	training's auc: 0.815788	valid_1's auc: 0.782386
[458]	training's auc: 0.815831	valid_1's auc: 0.78241
[459]	training's auc: 0.81588	valid_1's auc: 0.782432
[460]	training's auc: 0.815948	valid_1's auc: 0.78239
[461]	training's auc: 0.815992	valid_1's auc: 0.782402
[462]	training's auc: 0.816038	valid_1's auc: 0.782388
[463]	training's auc: 0.816145	valid_1's auc: 0.782365
[464]	training's auc: 0.816195	valid_1's auc: 0.782318
[465]	training's auc: 0.81627	valid_1's auc: 0.78236
[466]	training's auc: 0.816367	valid_1's auc: 0.782359
[467]	training's auc: 0.816454	valid_1's auc: 0.782382
[468]	training's auc: 0.816479	valid_1's auc: 0.782353
[469]	training's auc: 0.816554	valid_1's auc: 0.782321
[470]	training'

In [17]:
# bst = lgb.train(params, trn_lgb, 1000, valid_sets=[trn_lgb, val_lgb], early_stopping_rounds=20,
#                 init_model=bst, 
#                 learning_rates=lambda iter: 0.1 * (0.99 ** iter))

In [18]:
imp_df = pd.DataFrame([bst.feature_importance()], columns=feature_columns, index=['importance']).T.sort_values(by='importance', ascending=False)
imp_df

,importance
EXT_SOURCE_3,333
EXT_SOURCE_2,288
EXT_SOURCE_1,230
DAYS_BIRTH,177
AMT_ANNUITY,138
AMT_CREDIT,137
AMT_GOODS_PRICE,130
pcblc_CNT_INSTALMENT_FUTURE,121
DAYS_EMPLOYED,117
prevapp_AMT_ANNUITY,108


## predict

In [19]:
res_df = pd.DataFrame(bst.predict(X_tst), columns=[output_label_col])
res_df[output_id_col] = tst_df[output_id_col].values
res_df[[output_id_col, output_label_col]].to_csv(submission_path, index=False)